In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os

# ========================
# Setup 
# ========================
GRID_SIZE = 10          # 10x10 клітинок
SCREEN_W, SCREEN_H = 1366, 768  # Розмір екрану (для демонстрації клітинок)
SAVE_DIR = "./gaze_dataset"

CAP_WIDTH = 1280 # camera w
CAP_HEIGHT = 720 # camera h

# === Групи лендмарків FaceMesh (індекси) ===
LEFT_EYE_IDX = [33, 133, 159, 145, 160, 161, 246]  
LEFT_IRIS = [468, 469, 470, 471, 472]
RIGHT_EYE_IDX = [362, 263, 386, 374, 385, 390, 466]
RIGHT_IRIS = [473, 474, 475, 476, 477]
REFERENCE_IDX = [1, 6, 168, 55, 285, 152, 234, 454]  # nose + brow + chin

fixed_indices = LEFT_EYE_IDX + LEFT_IRIS + RIGHT_EYE_IDX + RIGHT_IRIS + REFERENCE_IDX

os.makedirs(SAVE_DIR, exist_ok=True)

# ========================
# Init Mediapipe
# ========================
model_path = r"./face_landmarker.task"

with open(model_path, "rb") as f:
    model_buffer = f.read()

# === Setup MediaPipe ===
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_buffer=model_buffer),
    running_mode=VisionRunningMode.IMAGE,
    num_faces=1,
)
detector = FaceLandmarker.create_from_options(options)



# === Capture frames from webcam ===
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAP_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAP_HEIGHT)
cap = cv2.VideoCapture(0)


# =========================
# Main collector of samples
# =========================
for row in range(GRID_SIZE):
    for col in range(GRID_SIZE):
        print(f"CELL >>>> ({row}, {col})")
        # time coundown for 1 cell gaze
        preview_time = 2.0  
        start_time = time.time()

        while time.time() - start_time < preview_time:
            ret, frame = cap.read()
            # cap.release()
            if not ret:
                continue
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
            result = detector.detect(mp_image)

            # 2D array for save frame without RGB, only landmarks
            frame_map = np.zeros((CAP_HEIGHT, CAP_WIDTH), dtype=np.uint8)

            if result.face_landmarks:
                face = result.face_landmarks[0]
                for idx in fixed_indices:
                    lm = face[idx]
                    x = int(lm.x * CAP_WIDTH)
                    y = int(lm.y * CAP_HEIGHT)
                    if 0 <= x < CAP_WIDTH and 0 <= y < CAP_HEIGHT:
                        frame_map[y, x] = 255  # one pixel for one landmark on 2D frame
                
                # Save frame_map
                filename = os.path.join(SAVE_DIR, f"cell_{row}_{col}_{int(time.time()*1000)}.npy")
                np.save(filename, frame_map)
cap.release()


CELL >>>> (0, 0)
CELL >>>> (0, 1)
CELL >>>> (0, 2)
CELL >>>> (0, 3)
CELL >>>> (0, 4)
CELL >>>> (0, 5)
CELL >>>> (0, 6)
CELL >>>> (0, 7)
CELL >>>> (0, 8)
CELL >>>> (0, 9)
CELL >>>> (1, 0)
CELL >>>> (1, 1)
CELL >>>> (1, 2)
CELL >>>> (1, 3)
CELL >>>> (1, 4)
CELL >>>> (1, 5)
CELL >>>> (1, 6)


KeyboardInterrupt: 